#Preprocesmamiento de datos.

En esta parte se selecciona el conjunto de datos y se realiza construye un corpus para poder trabajar de forma adecuada en nuestro proyecto.


In [ ]:
!pip3 install html2text 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def convierteFecha(fecha):
  fecha = fecha.split("T")
  fecha[1] = fecha[1].replace('.',' ')
  fecha[1] = fecha[1].split(" ")
  date = fecha[0]
  hora = fecha[1][0]
  fechaFin = date + " " + hora
  return fechaFin 

In [ ]:
import pandas as pd
from xml.dom import minidom
import html2text

h = html2text.HTML2Text()

h.ignore_links = True
h.ignore_images = True

try:
  doc1 = minidom.parse("Posts1.xml")
  doc2 = minidom.parse("Posts2.xml")
except:
   print("error")

df = pd.DataFrame(columns=['Id', 'PostTypeId','AcceptedAnswerId', 'ParentId','Title','Body','CreationDate','Score','AnswerCount','Archivo'])

rows1 = doc1.getElementsByTagName("row")
rows2 = doc2.getElementsByTagName("row")
#rows = rows1 + rows2
for row in rows1:
    id = row.getAttribute("Id")
    PostTypeId = row.getAttribute("PostTypeId")
    CreationDate = row.getAttribute("CreationDate")
    Body = row.getAttribute("Body")
    Title = row.getAttribute("Title")
    ParentId = row.getAttribute("ParentId")
    AcceptedAnswerId = row.getAttribute("AcceptedAnswerId")
    Score = row.getAttribute("Score")
    AnswerCount = row.getAttribute("AnswerCount")
    Archivo = 'Posts1'
    df = df.append({'Id': id, 'PostTypeId':PostTypeId, 'AcceptedAnswerId':AcceptedAnswerId,
                    'ParentId':ParentId,'Title':Title, 'Body': h.handle(Body) ,'CreationDate':convierteFecha(CreationDate),'AnswerCount': AnswerCount,'Archivo': Archivo,'Score':Score }, ignore_index=True)

for row in rows2:
    id = row.getAttribute("Id")
    PostTypeId = row.getAttribute("PostTypeId")
    CreationDate = row.getAttribute("CreationDate")
    Body = row.getAttribute("Body")
    Title = row.getAttribute("Title")
    ParentId = row.getAttribute("ParentId")
    AcceptedAnswerId = row.getAttribute("AcceptedAnswerId")
    Score = row.getAttribute("Score")
    AnswerCount = row.getAttribute("AnswerCount")
    Archivo = 'Posts2'
    df = df.append({'Id': id, 'PostTypeId':PostTypeId, 'AcceptedAnswerId':AcceptedAnswerId,
                    'ParentId':ParentId,'Title':Title, 'Body': h.handle(Body) ,'CreationDate':convierteFecha(CreationDate),'AnswerCount': AnswerCount,'Archivo': Archivo,'Score':Score }, ignore_index=True)


df['Body'] = df['Body'].apply(lambda x: x.replace('\n',''))

df.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,Title,Body,CreationDate,Score,AnswerCount,Archivo
0,1,1,3,,"What is ""backprop""?","What does ""backprop"" mean? Is the ""backprop"" t...",2016-08-02 15:39:14,10,5,Posts1
1,2,1,9,,How does noise affect generalization?,Does increasing the noise in data help to impr...,2016-08-02 15:40:20,14,3,Posts1
2,3,2,,1,,"""Backprop"" is the same as ""backpropagation"": i...",2016-08-02 15:40:24,15,,Posts1
3,4,1,12,,How to find the optimal number of neurons per ...,"When you're writing your algorithm, how do you...",2016-08-02 15:41:22,33,4,Posts1
4,6,1,20,,Are humans intelligent according to the defini...,Given the following definition of an intellige...,2016-08-02 15:43:35,7,2,Posts1


##Creación del diccionario.

In [ ]:
#Separamos primero en preguntas y respuestas de la forma que queremos, luego las juntamos las que compartan el mismo parentID y provengan del mismo archivo

#Hacemos un diccionario de preguntas con el atributo respuestas vacio 
dic = []

for i in range(0,len(df)):
  if df['PostTypeId'][i] == '1':
    dic.append({'Id':df['Id'][i],'Title':df['Title'][i],'Pregunta':df['Body'][i],'AcceptedId':df['AcceptedAnswerId'][i], 'Date':df['CreationDate'][i], 'Respuestas':[], 'AnswerCount':df['AnswerCount'][i], 'Archivo':df['Archivo'][i]})


#Hacemos un diccionario de respuestas 
dicR = []

for i in range(0,len(df)):
   if df['PostTypeId'][i] == '2':
     dicR.append({'Id':df['Id'][i],'ParentId': df['ParentId'][i], 'Date': df['CreationDate'][i], 'Body': df['Body'][i],'score': df['Score'][i], 'Archivo':df['Archivo'][i]})


#Unimos los dos diccionarios en uno solo
for i in range(0,len(dic)):
  for j in range(0,len(dicR)):
    if dicR[j]['ParentId'] == dic[i]['Id'] and dicR[j]['Archivo'] == dic[i]['Archivo']:
      dic[i]['Respuestas'].append(dicR[j])


In [38]:
scoreMax = 0
for i in range(0,len(dic)):
  scoreMax = 0
  if dic[i]['AcceptedId'] == '':
    for j in dic[i]['Respuestas']:
      if int(j['score']) > scoreMax: 
        scoreMax = int(j['score'])
    dic[i]['score_max']=scoreMax    

In [ ]:
for i in range(0,len(dic)):
  if dic[i]['AcceptedId'] == '':
    for j in dic[i]['Respuestas']: 
      if j['score'] == str(dic[i]['score_max']):
        dic[i]['AcceptedId'] = j['Id']

In [ ]:
import time
from datetime import date

for i in range(0,len(dic)):
  fecha = dic[i]['Date'].split(" ")
  dic[i]['Date'] = time.strptime(dic[i]['Date'], "%Y-%m-%d %H:%M:%S")

In [ ]:
#Eliminamos puntuación, urls y se escribe el texto en letra minúscula.

import string
import re 
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

def remove_urls (texto):
   texto = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', texto) 
   return texto 

for i in range(0,len(dic)):
  dic[i]['Pregunta'] = remove_urls(dic[i]['Pregunta'])
  dic[i]['Pregunta'] = remove_punctuation(dic[i]['Pregunta'])
  dic[i]['Pregunta'] = dic[i]['Pregunta'].lower().strip()
  for j in range(0,len(dic[i]['Respuestas'])):
    dic[i]['Respuestas'][j]['Body'] = remove_urls(dic[i]['Respuestas'][j]['Body'])
    dic[i]['Respuestas'][j]['Body'] = remove_punctuation(dic[i]['Respuestas'][j]['Body'])
    dic[i]['Respuestas'][j]['Body'] = dic[i]['Respuestas'][j]['Body'].lower().strip()

dic[0]

In [ ]:
#Se unen todos los textos de las preguntas y sus respuestas en un campo para trabajar fácilmente 
for i in range(0,len(dic)):
  textos = []
  textos.append(dic[i]['Pregunta'])
  for j in range(0,len(dic[i]['Respuestas'])):
    textos.append(dic[i]['Respuestas'][j]['Body'])
  dic[i]['Textos'] = textos

In [ ]:
#Contar el número de preguntas y respuestas.
print("Tenemos", len(dic) ,"preguntas.") 
respuestas = 0
for i in range(0,len(dic)):
  respuestas = respuestas + len(dic[i]['Respuestas'])

print("Tenemos", respuestas ,"respuestas.")


Tenemos 14777 preguntas.
Tenemos 15982 respuestas.


In [ ]:
#Seleccionamos preguntas con cuatro o más respuestas. 
copia = []
for i in range(0,len(dic)):
  if len(dic[i]['Respuestas']) > 3: 
    copia.append(dic[i])

print("Tenemos", len(copia) ,"preguntas.") 
respuestas = 0
for i in range(0,len(copia)):
  respuestas = respuestas + len(copia[i]['Respuestas'])

print("Tenemos", respuestas ,"respuestas.")


Tenemos 395 preguntas.
Tenemos 1960 respuestas.


In [ ]:
#Para exportar el conjunto una vez procesado: 
import json
tf = open("corpusProcesado.json", "w")
json.dump(copia,tf)
tf.close()

In [ ]:
#Para no hacer el preprocesamiento se puede ejecutar desde aquí, siempre y cuando esté cargado el archivo "corpusProcesado.json"
tf = open("corpusProcesado.json", "r")
copia = json.load(tf)

#Modelos

##Primer modelo
TFIDF + Similitud del coseno

In [ ]:
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfTransformer
# Descargamos las stopwords de NLTK
# Si no tenemos instalado NLTK lo instalamos
# !pip3 install -U nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Calculamos la similitud usando TFIDF
dic1 = copia
count_vect = CountVectorizer(stop_words=stopwords, ngram_range=(1,2))
for i in range(0,len(dic1)):
  X_counts = count_vect.fit_transform(dic1[i]['Textos'])
  # Calculamos ahora el TFIDF
  tfidf_transformer = TfidfTransformer()
  X_TFIDF = tfidf_transformer.fit_transform(X_counts)

  # Calculamos la similitud de los documentos con el coseno para TFIDF
  results = cosine_similarity(X_TFIDF[1::],X_TFIDF[0]).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

  #Vamos ahora a calcular los maximos correspondientes para asignarlos en el campo prediccion para cada pregunta
  dic1[i]['Pred'] = dic1[i]['Respuestas'][np.where(results == np.amax(results))[0][0]]['Id']



In [ ]:
#Funciones para el cálculo del recall precision y f1
def recall(VP, FN):
  rec = VP / (VP+FN)
  return rec

def precision(VP, FP):
  prec = VP / (VP + FP)
  return prec

def accuracy(VP, FN, FP, VN):
  N = VP+FN+FP+VN
  return (VP+VN)/N

def fscore(prec, recall):
  num = 2 * prec * recall
  den = prec + recall
  return num / den

In [ ]:
#Una vez calculadas las predicciones hacemos la matriz de confusión para el cálculo de los estadísticos
dic = dic1 
VP = 0
FN = 0
FP = 0
VN = 0
for i in range(0,len(dic)):
  for j in range(0,len(dic[i]['Respuestas'])):
    if dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      VP = VP + 1
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      FP = FP + 1 
    elif dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      FN = FN + 1 
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      VN = VN + 1

print("VP ="+str(VP) + "    " + "FN ="+str(FN))
print("FP ="+str(FP) + "    " + "VN ="+str(VN))

VP =108    FN =287
FP =287    VN =1278


In [ ]:
#Medidas y estadísticos del primer modelo: 
print("Recall: " + str(recall(VP,FN)))
print("Precision: "+ str(precision(VP,FP)))
print("Accuracy: " + str(accuracy(VP, FN, FP, VN)))
print("f1-score: " + str(fscore(precision(VP,FP), recall(VP,FN))))

Recall: 0.27341772151898736
Precision: 0.27341772151898736
Accuracy: 0.7071428571428572
f1-score: 0.27341772151898736


##Segundo modelo
Por fecha más actual

In [ ]:
#Se asigna como prediccion la fecha de la primera respuesta, se recorren todas las respuestas y en caso de encontrar una respuesta con fecha más actual se modifica.
dic2 = copia
for i in range(0, len(copia)):
  dic2[i]['Pred'] = dic2[i]['Respuestas'][0]['Id'] 
  k=0
  for j in range(0,len(dic2[i]['Respuestas'])-1):
    fechaPred = dic2[i]['Respuestas'][k]['Date']
    if fechaPred > dic2[i]['Respuestas'][j+1]['Date']:
      dic2[i]['Pred'] = dic2[i]['Respuestas'][j+1]['Id']
      k = j

#Por ver un ejemplo
dic2[0]

In [ ]:

dic = dic2
VP = 0
FN = 0
FP = 0
VN = 0
for i in range(0,len(dic)):
  for j in range(0,len(dic[i]['Respuestas'])):
    if dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      VP = VP + 1
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      FP = FP + 1 
    elif dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      FN = FN + 1 
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      VN = VN + 1

print("VP ="+str(VP) + "    " + "FN ="+str(FN))
print("FP ="+str(FP) + "    " + "VN ="+str(VN))

VP =124    FN =271
FP =271    VN =1294


In [ ]:
#Medidas y estadísticos del segundo modelo: 
prec2 = precision(VP,FP)
recall2 = recall(VP,FN)
print("Recall: " + str(recall2))
print("Precision: "+ str(prec2))
print("Accuracy: " + str(accuracy(VP, FN, FP, VN)))
print("f1-score: " + str(fscore(prec2, recall2)))

Recall: 0.3139240506329114
Precision: 0.3139240506329114
Accuracy: 0.7234693877551021
f1-score: 0.3139240506329114


##Tercer modelo

BERT

In [ ]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 49.1 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers

#Cargamos el tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")


#Cargamos el modelo pre-entrenado
bert_model = transformers.AutoModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

def bert_sentence_embeddings(texts, model):
  
  def mean_pooling (model_output, attention_mask):
    
    token_embeddings = model_output[0]
    
    input_mask_expanded = attention_mask.unsqueeze (-1).expand (token_embeddings.size ()).float ()
    
    sum_embeddings = torch.sum (token_embeddings * input_mask_expanded, 1)

    sum_mask = torch.clamp (input_mask_expanded.sum (1), min = 1e-9)

    return sum_embeddings / sum_mask
    
    
  #Cargamos el modelo y el tokenizador
  tokenizer = transformers.AutoTokenizer.from_pretrained ("bert-base-cased")
  model = transformers.AutoModel.from_pretrained ("bert-base-cased")


  #Codificación de los valores de entrada
  encoded_inputs = tokenizer (
      texts, 
      padding = True, 
      truncation = True, 
      max_length = 256, 
      return_tensors = 'pt'
  )
        
        
  # Construimos los embeddings
  with torch.no_grad ():
      model_outputs = model (**encoded_inputs)
      sentence_embeddings = mean_pooling (model_outputs, encoded_inputs['attention_mask'])
      a = pd.DataFrame (sentence_embeddings.cpu ().numpy ())
            
  return a

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import scipy.sparse as sp
from transformers import logging

dic3 = copia
# Calculamos la similitud usando sentence embeddings
for i in range(0, len(dic3)):
  sentence_embeddings = bert_sentence_embeddings(dic3[i]['Textos'], bert_model)
  logging.set_verbosity_error()
  if not sp.issparse(sentence_embeddings):
            sentence_embeddings = sp.csr_matrix(sentence_embeddings, dtype=np.float64)
  # Calculamos la similitud de los documentos con el coseno para sentence embeddings
  results = cosine_similarity(sentence_embeddings[1::],sentence_embeddings[0]).reshape(-1,) # Op -- (n_docs,1) -- Cosine Sim with each doc
  dic3[i]['Pred'] = dic3[i]['Respuestas'][np.where(results == np.amax(results))[0][0]]['Id']


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
dic = dic3
VP = 0
FN = 0
FP = 0
VN = 0
for i in range(0,len(dic)):
  for j in range(0,len(dic[i]['Respuestas'])):
    if dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      VP = VP + 1
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      FP = FP + 1 
    elif dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      FN = FN + 1 
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      VN = VN + 1

print("VP ="+str(VP) + "    " + "FN ="+str(FN))
print("FP ="+str(FP) + "    " + "VN ="+str(VN))

VP =110    FN =285
FP =285    VN =1280


In [ ]:
prec3 = precision(VP,FP)
recall3 = recall(VP,FN)
print("Recall: " + str(recall3))
print("Precision: "+ str(prec3))
print("Accuracy: " + str(accuracy(VP, FN, FP, VN)))
print("f1-score: " + str(fscore(prec3, recall3)))

Recall: 0.27848101265822783
Precision: 0.27848101265822783
Accuracy: 0.7091836734693877
f1-score: 0.27848101265822783


##Cuarto modelo 
FastText


In [ ]:
!pip3 install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.4 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3142688 sha256=f017f9a697fcaece6728483dd0dedae80f1d9059d74979c0cd2d56d621f0980c
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz

--2022-06-30 15:16:46--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  51.8MB/s    in 89s     

2022-06-30 15:18:16 (48.3 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [ ]:
ft_model = fasttext.load_model ("cc.en.300.bin")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import scipy.sparse as sp

dic4 = copia

import pandas as pd

for i in range(0,len(dic4)):
  df_textos = pd.DataFrame(dic4[i]['Textos'])
  # Calculamos la similitud usando sentence embeddings
  fasttext_sentence_embeddings = df_textos[0].apply(lambda x: ft_model.get_sentence_vector(x))
  fasttext_sentence_embeddings = fasttext_sentence_embeddings.to_list()
  if not sp.issparse(fasttext_sentence_embeddings):
              fasttext_sentence_embeddings = sp.csr_matrix(fasttext_sentence_embeddings, dtype=np.float64)
  # Calculamos la similitud de los documentos con el coseno para sentence embeddings
  results = cosine_similarity(fasttext_sentence_embeddings[1::],fasttext_sentence_embeddings[0]).reshape(-1,) # Op -- (n_docs,1) -- Cosine Sim with each doc  
  
  dic4[i]['Pred'] = dic4[i]['Respuestas'][np.where(results == np.amax(results))[0][0]]['Id']

In [ ]:
dic = dic4
VP = 0
FN = 0
FP = 0
VN = 0
for i in range(0,len(dic)):
  for j in range(0,len(dic[i]['Respuestas'])):
    if dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      VP = VP + 1
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] == dic[i]['Pred']:
      FP = FP + 1 
    elif dic[i]['AcceptedId'] == dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      FN = FN + 1 
    elif dic[i]['AcceptedId'] != dic[i]['Respuestas'][j]['Id'] and dic[i]['Respuestas'][j]['Id'] != dic[i]['Pred']:
      VN = VN + 1

print("VP ="+str(VP) + "    " + "FN ="+str(FN))
print("FP ="+str(FP) + "    " + "VN ="+str(VN))

VP =135    FN =260
FP =260    VN =1305


In [ ]:
prec4 = precision(VP,FP)
recall4 = recall(VP,FN)
print("Recall: " + str(recall4))
print("Precision: "+ str(prec4))
print("Accuracy: " + str(accuracy(VP, FN, FP, VN)))
print("f1-score: " + str(fscore(prec4, recall4)))

Recall: 0.34177215189873417
Precision: 0.34177215189873417
Accuracy: 0.7346938775510204
f1-score: 0.34177215189873417
